In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lungs-disease-data/data/test/Type 1 disease/0108.jpeg
/kaggle/input/lungs-disease-data/data/test/Type 1 disease/0119.jpeg
/kaggle/input/lungs-disease-data/data/test/Type 1 disease/0115.jpeg
/kaggle/input/lungs-disease-data/data/test/Type 1 disease/auntminnie-2020_01_31_20_24_2322_2020_01_31_x-ray_coronavirus_US.jpg
/kaggle/input/lungs-disease-data/data/test/Type 1 disease/COVID-00037.jpg
/kaggle/input/lungs-disease-data/data/test/Type 1 disease/0106.jpeg
/kaggle/input/lungs-disease-data/data/test/Type 1 disease/0111.jpg
/kaggle/input/lungs-disease-data/data/test/Type 1 disease/COVID-00022.jpg
/kaggle/input/lungs-disease-data/data/test/Type 1 disease/auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg
/kaggle/input/lungs-disease-data/data/test/Type 1 disease/0120.jpg
/kaggle/input/lungs-disease-data/data/test/Type 1 disease/094.png
/kaggle/input/lungs-disease-data/data/test/Type 1 disease/COVID-00003b.jpg
/kaggle/input/lungs-disease-data/data/test/Type 1

In [2]:
import os
import cv2
import pickle
import random as rnd
import numpy as np
import pandas as pd

In [3]:
def preprocess(image_path, image_size):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (image_size, image_size))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image/255
    return image

In [4]:
def create_dataframe(datadir):
    result_dataframe = list()
    for train_test_name in os.listdir(datadir):
        label_dir = os.path.join(datadir, train_test_name)
        for label_name in os.listdir(label_dir):
            image_dir = os.path.join(label_dir, label_name)
            for image_path in os.listdir(image_dir):
                record_dict = dict()
                if train_test_name == 'test':
                    record_dict['is_train'] = 0
                    record_dict['label'] = label_name.replace('healthy', 'Healthy')
                    record_dict['image_path'] = image_dir + image_path
                    
                else:
                    record_dict['is_train'] = 1
                    record_dict['label'] = label_name
                    
                    record_dict['image_path'] = image_dir + '/' + image_path
                result_dataframe.append(record_dict)
    return result_dataframe

In [5]:
result_dataframe = create_dataframe('/kaggle/input/lungs-disease-data/data')
result_dataframe = pd.DataFrame.from_records(result_dataframe)
result_dataframe

,is_train,label,image_path
0,0,Type 1 disease,/kaggle/input/lungs-disease-data/data/test/Typ...
1,0,Type 1 disease,/kaggle/input/lungs-disease-data/data/test/Typ...
2,0,Type 1 disease,/kaggle/input/lungs-disease-data/data/test/Typ...
3,0,Type 1 disease,/kaggle/input/lungs-disease-data/data/test/Typ...
4,0,Type 1 disease,/kaggle/input/lungs-disease-data/data/test/Typ...
...,...,...,...
312,1,Type 2 disease,/kaggle/input/lungs-disease-data/data/train/Ty...
313,1,Type 2 disease,/kaggle/input/lungs-disease-data/data/train/Ty...
314,1,Type 2 disease,/kaggle/input/lungs-disease-data/data/train/Ty...
315,1,Type 2 disease,/kaggle/input/lungs-disease-data/data/train/Ty...


In [6]:
train_dataframe, test_dataframe = result_dataframe[result_dataframe['is_train'] == 1].drop('is_train', axis=1), result_dataframe[result_dataframe['is_train'] == 0].drop('is_train', axis=1)
train_dataframe

,label,image_path
66,Type 1 disease,/kaggle/input/lungs-disease-data/data/train/Ty...
67,Type 1 disease,/kaggle/input/lungs-disease-data/data/train/Ty...
68,Type 1 disease,/kaggle/input/lungs-disease-data/data/train/Ty...
69,Type 1 disease,/kaggle/input/lungs-disease-data/data/train/Ty...
70,Type 1 disease,/kaggle/input/lungs-disease-data/data/train/Ty...
...,...,...
312,Type 2 disease,/kaggle/input/lungs-disease-data/data/train/Ty...
313,Type 2 disease,/kaggle/input/lungs-disease-data/data/train/Ty...
314,Type 2 disease,/kaggle/input/lungs-disease-data/data/train/Ty...
315,Type 2 disease,/kaggle/input/lungs-disease-data/data/train/Ty...


In [7]:
train_dataframe['label'].value_counts()

label
Type 1 disease    111
Healthy            70
Type 2 disease     70
Name: count, dtype: int64

In [8]:
test_dataframe['label'].value_counts()

label
Type 1 disease    26
Healthy           20
Type 2 disease    20
Name: count, dtype: int64

In [9]:
test_dataframe['label'].value_counts().keys() == train_dataframe['label'].value_counts().keys()

array([ True,  True,  True])

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

train_dataframe['label'] = le.fit_transform(train_dataframe['label'].values)
test_dataframe['label'] = le.transform(test_dataframe['label'].values)

In [11]:
from sklearn.model_selection import train_test_split

train_images, validation_images, train_labels, validation_labels = train_test_split(train_dataframe['image_path'].values, train_dataframe['label'].values, test_size=0.2,stratify=train_dataframe['label'].values, random_state=42,shuffle=True)
test_images, test_labels = test_dataframe['image_path'].values, test_dataframe['label'].values

In [12]:
print(train_images.shape, train_labels.shape)
print(validation_images.shape, validation_labels.shape)
print(test_images.shape, test_labels.shape)

(200,) (200,)
(51,) (51,)
(66,) (66,)


In [13]:
print(np.unique(train_labels,return_counts=True))
print(np.unique(validation_labels,return_counts=True))
print(np.unique(test_labels,return_counts=True))

(array([0, 1, 2]), array([56, 88, 56]))
(array([0, 1, 2]), array([14, 23, 14]))
(array([0, 1, 2]), array([20, 26, 20]))


In [14]:
!git clone https://github.com/facebookresearch/dinov2
%cd dinov2
!pip install -r requirements.txt -r requirements-extras.txt

Cloning into 'dinov2'...
remote: Enumerating objects: 485, done.
remote: Counting objects: 100% (260/260), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 485 (delta 172), reused 160 (delta 160), pack-reused 225
Receiving objects: 100% (485/485), 1.11 MiB | 13.20 MiB/s, done.
Resolving deltas: 100% (240/240), done.
/kaggle/working/dinov2
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 481.4 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 35.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 1.1 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━

In [15]:
import os
import random
import zipfile
from copy import deepcopy
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import torch
import warnings
from PIL import Image
from torch import nn, optim
from torchvision import datasets, transforms
from dinov2.models.vision_transformer import vit_small, vit_base, vit_large, vit_giant2
import pandas as pd
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

/kaggle/working/dinov2/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/kaggle/working/dinov2/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/kaggle/working/dinov2/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [57]:
from PIL import Image

class DiseaseClassification(Dataset):

    def __init__(self, X, Y, transform=None):
        self.X = X
        self.Y = Y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.X[idx]
        image = Image.open(img_name)
        image = image.convert('RGB')
        #image = image / 255
        image_class = self.Y[idx]
        if self.transform:
            image = self.transform(image)

        return image, image_class

In [58]:
class ResizeAndPad:
    def __init__(self, target_size, multiple):
        self.target_size = target_size
        self.multiple = multiple

    def __call__(self, img):
        # Resize the image
        img = transforms.Resize(self.target_size)(img)

        # Calculate padding
        pad_width = (self.multiple - img.width % self.multiple) % self.multiple
        pad_height = (self.multiple - img.height % self.multiple) % self.multiple

        # Apply padding
        img = transforms.Pad((pad_width // 2, pad_height // 2, pad_width - pad_width // 2, pad_height - pad_height // 2))(img)

        return img

image_dimension = 526

# This is what DinoV2 sees
target_size = (image_dimension, image_dimension)

# Below are functions that every image will be passed through, including data augmentations
data_transforms = {
    "train": transforms.Compose(
        [
            ResizeAndPad(target_size, 14),
            transforms.RandomAdjustSharpness(2, 0.7),
            transforms.RandomRotation(360),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    ),
    "test": transforms.Compose([ ResizeAndPad(target_size, 14),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                              ]
    )
  }

In [59]:
train_dataset = DiseaseClassification(train_images, train_labels, data_transforms["train"])
valdiation_dataset = DiseaseClassification(validation_images, validation_labels, data_transforms["test"])
test_dataset = DiseaseClassification(test_images, test_labels, data_transforms["test"])


train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
validation_dataloader = torch.utils.data.DataLoader(valdiation_dataset, batch_size=8, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=True)

In [60]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [61]:
!wget https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_reg4_pretrain.pth

--2024-05-07 11:49:14--  https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_reg4_pretrain.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.164.78.121, 18.164.78.72, 18.164.78.81, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.164.78.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88291785 (84M) [binary/octet-stream]
Saving to: 'dinov2_vits14_reg4_pretrain.pth.5'

dinov2_vits14_reg4_ 100%[===================>]  84.20M   239MB/s    in 0.4s    

2024-05-07 11:49:14 (239 MB/s) - 'dinov2_vits14_reg4_pretrain.pth.5' saved [88291785/88291785]



In [62]:

# Define a new classifier layer that contains a few linear layers with a ReLU to make predictions positive
class DinoVisionTransformerClassifier(nn.Module):

    def __init__(self, model_size="small"):
        super(DinoVisionTransformerClassifier, self).__init__()
        self.model_size = model_size

        # loading a model with registers
        n_register_tokens = 4

        if model_size == "small":
            model = vit_small(patch_size=14,
                              img_size=526,
                              init_values=1.0,
                              num_register_tokens=n_register_tokens,
                              block_chunks=0)
            self.embedding_size = 384
            self.number_of_heads = 6
            local_directory = 'dinov2_vits14_reg4_pretrain.pth'

        elif model_size == "base":
            model = vit_base(patch_size=14,
                             img_size=526,
                             init_values=1.0,
                             num_register_tokens=n_register_tokens,
                             block_chunks=0)
            self.embedding_size = 768
            self.number_of_heads = 12

        elif model_size == "large":
            model = vit_large(patch_size=14,
                              img_size=526,
                              init_values=1.0,
                              num_register_tokens=n_register_tokens,
                              block_chunks=0)
            self.embedding_size = 1024
            self.number_of_heads = 16

        elif model_size == "giant":
            model = vit_giant2(patch_size=14,
                               img_size=526,
                               init_values=1.0,
                               num_register_tokens=n_register_tokens,
                               block_chunks=0)
            self.embedding_size = 1536
            self.number_of_heads = 24

        # Download pre-trained weights and place locally as-needed:
        # - small: https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_reg4_pretrain.pth
        # - base:  https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb14/dinov2_vitb14_reg4_pretrain.pth
        # - large: https://dl.fbaipublicfiles.com/dinov2/dinov2_vitl14/dinov2_vitl14_reg4_pretrain.pth
        # - giant: https://dl.fbaipublicfiles.com/dinov2/dinov2_vitg14/dinov2_vitg14_reg4_pretrain.pth
        model.load_state_dict(torch.load(local_directory))

        self.transformer = deepcopy(model)

        self.classifier = nn.Sequential(nn.Linear(self.embedding_size, 256), nn.ReLU(), nn.Linear(256, 3))

    def forward(self, x):
        x = self.transformer(x)
        x = self.transformer.norm(x)
        x = self.classifier(x)
        return x


In [63]:
criterion = nn.CrossEntropyLoss()
num_epochs = 100

In [64]:
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
from tqdm import tqdm

model = DinoVisionTransformerClassifier("small")
model = model.to(device)
model = model.train()

optimizer = optim.Adam(model.parameters(), lr=1e-6)

train_epoch_losses, test_epoch_losses = [], []
print("Training...")
for epoch in tqdm(range(num_epochs)):
    train_batch_accuracies, test_batch_accuracies = [], []
    train_batch_y_hat, train_batch_y = [], []
    test_batch_y_hat, test_batch_y = [], []

    train_batch_losses, test_batch_losses = [], []
    for data in train_dataloader:
        # get the input batch and the labels
        batch_of_images, labels = data
        labels = labels.type(torch.LongTensor)

        # zero the parameter gradients
        optimizer.zero_grad()

        # model prediction
        output = model(batch_of_images.to(device)).squeeze(dim=1)

        # compute loss and do gradient descent
        loss = criterion(output, labels.to(device))
        train_batch_losses.append(loss.item())

        loss.backward()
        optimizer.step()


        # Calculate and record batch accuracy
        y_hat = torch.max(output.data,1).indices
        train_batch_y_hat.append(y_hat.cpu().numpy())
        train_batch_y.append(labels.cpu().numpy())

    model.eval()
    with torch.no_grad():
        for data in validation_dataloader:
          # get the input batch and the labels
            batch_of_images, labels = data
            labels = labels.type(torch.LongTensor)

          # model prediction
            output = model(batch_of_images.to(device)).squeeze(dim=1)

            loss = criterion(output, labels.to(device))
            test_batch_losses.append(loss.item())

            y_hat = torch.max(output.data,1).indices
            test_batch_y_hat.append(y_hat.cpu().numpy())
            test_batch_y.append(labels.cpu().numpy())


    train_epoch_losses.append(np.mean(train_batch_losses))
    test_epoch_losses.append(np.mean(test_batch_losses))

    test_epoch_y_hat = [item for row in test_batch_y_hat for item in row]
    test_epoch_y = [item for row in test_batch_y for item in row]


    train_epoch_y_hat = [item for row in train_batch_y_hat for item in row]
    train_epoch_y = [item for row in train_batch_y for item in row]
    
    
    train_f1 = f1_score(train_epoch_y_hat, train_epoch_y, average = 'macro')
    train_precision = precision_score(train_epoch_y_hat, train_epoch_y, average = 'macro')
    train_recall = recall_score(train_epoch_y_hat, train_epoch_y, average = 'macro')
    train_cr = classification_report(train_epoch_y_hat, train_epoch_y)

    test_f1 = f1_score(test_epoch_y_hat, test_epoch_y, average = 'macro')
    test_precision = precision_score(test_epoch_y_hat, test_epoch_y, average = 'macro')
    test_recall = recall_score(test_epoch_y_hat, test_epoch_y, average = 'macro')
    test_cr = classification_report(test_epoch_y_hat, test_epoch_y)



    print("Epoch {}: Train F1 = {:.3f}%, Train Precision = {:.3f}, Train Recall = {:.3f} \n Test F1 = {:.3f}, Test Precision = {:.3f}, Test Recall = {:.3f}".format(epoch, train_f1, train_precision, train_recall, test_f1, test_precision, test_recall))
    print('Train Classification Report')
    print(train_cr)
    print('Test Classification Report')
    print(test_cr)
    print("-----------")

Training...


  0%|          | 0/100 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_class

Epoch 0: Train F1 = 0.322%, Train Precision = 0.414, Train Recall = 0.288 
 Test F1 = 0.252, Test Precision = 0.391, Test Recall = 0.433
Train Classification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.42      0.49      0.45        75
           2       0.82      0.37      0.51       124

    accuracy                           0.41       200
   macro avg       0.41      0.29      0.32       200
weighted avg       0.67      0.41      0.49       200

Test Classification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.17      1.00      0.30         4
           2       1.00      0.30      0.46        47

    accuracy                           0.35        51
   macro avg       0.39      0.43      0.25        51
weighted avg       0.94      0.35      0.45        51

-----------


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarni

Epoch 1: Train F1 = 0.339%, Train Precision = 0.399, Train Recall = 0.612 
 Test F1 = 0.354, Test Precision = 0.464, Test Recall = 0.444
Train Classification Report
              precision    recall  f1-score   support

           0       0.07      0.80      0.13         5
           1       0.97      0.47      0.64       179
           2       0.16      0.56      0.25        16

    accuracy                           0.49       200
   macro avg       0.40      0.61      0.34       200
weighted avg       0.88      0.49      0.59       200

Test Classification Report
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.39      1.00      0.56         9
           2       1.00      0.33      0.50        42

    accuracy                           0.45        51
   macro avg       0.46      0.44      0.35        51
weighted avg       0.89      0.45      0.51        51

-----------


...

Epoch 97: Train F1 = 1.000%, Train Precision = 1.000, Train Recall = 1.000 
 Test F1 = 0.957, Test Precision = 0.962, Test Recall = 0.954
Train Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        56
           1       1.00      1.00      1.00        88
           2       1.00      1.00      1.00        56

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200

Test Classification Report
              precision    recall  f1-score   support

           0       0.93      0.93      0.93        14
           1       0.96      1.00      0.98        22
           2       1.00      0.93      0.97        15

    accuracy                           0.96        51
   macro avg       0.96      0.95      0.96        51
weighted avg       0.96      0.96      0.96        51

-----------


 99%|█████████▉| 99/100 [1:11:10<00:43, 43.01s/it]

Epoch 98: Train F1 = 1.000%, Train Precision = 1.000, Train Recall = 1.000 
 Test F1 = 0.957, Test Precision = 0.962, Test Recall = 0.954
Train Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        56
           1       1.00      1.00      1.00        88
           2       1.00      1.00      1.00        56

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200

Test Classification Report
              precision    recall  f1-score   support

           0       0.93      0.93      0.93        14
           1       0.96      1.00      0.98        22
           2       1.00      0.93      0.97        15

    accuracy                           0.96        51
   macro avg       0.96      0.95      0.96        51
weighted avg       0.96      0.96      0.96        51

-----------


100%|██████████| 100/100 [1:11:53<00:00, 43.13s/it]

Epoch 99: Train F1 = 1.000%, Train Precision = 1.000, Train Recall = 1.000 
 Test F1 = 0.957, Test Precision = 0.962, Test Recall = 0.954
Train Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        56
           1       1.00      1.00      1.00        88
           2       1.00      1.00      1.00        56

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200

Test Classification Report
              precision    recall  f1-score   support

           0       0.93      0.93      0.93        14
           1       0.96      1.00      0.98        22
           2       1.00      0.93      0.97        15

    accuracy                           0.96        51
   macro avg       0.96      0.95      0.96        51
weighted avg       0.96      0.96      0.96        51

-----------


#### Best Iteration

Epoch 37: Train F1 = 0.984%, Train Precision = 0.984, Train Recall = 0.984 
 Test F1 = 0.981, Test Precision = 0.986, Test Recall = 0.978
Train Classification Report
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        56
           1       0.99      0.99      0.99        88
           2       0.98      0.98      0.98        56

    accuracy                           0.98       200
   macro avg       0.98      0.98      0.98       200
weighted avg       0.98      0.98      0.98       200

Test Classification Report
              precision    recall  f1-score   support

           0       1.00      0.93      0.97        15
           1       0.96      1.00      0.98        22
           2       1.00      1.00      1.00        14

    accuracy                           0.98        51
   macro avg       0.99      0.98      0.98        51
weighted avg       0.98      0.98      0.98        51